In [184]:
import pandas as pd
import numpy as np
import nltk
import sys
import xgboost
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,accuracy_score
import pickle

In [186]:
file=pd.read_csv("IMDB Dataset.csv.zip")

In [187]:
file.shape

(50000, 2)

In [190]:
file.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [192]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [194]:
file['sentiment'] = le.fit_transform(file['sentiment'])

In [196]:
file.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1


In [198]:
# First, import nltk and download the required resources

nltk.download('punkt')  # Download the punkt tokenizer data
nltk.download('stopwords')  # Download stopwords

# Define the remove_stopwords function
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    
    if not isinstance(text, str):
        text = str(text)
    
    # Tokenize and remove stopwords
    try:
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
        # Return the filtered text as a string
        return ' '.join(filtered_text)
    except LookupError:
        # If there's still an issue with the tokenizer, try a simpler approach
        print("Warning: Using simple tokenization as fallback")
        words = text.split()
        filtered_text = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_text)



[nltk_data] Downloading package punkt to C:\Users\NIKITA
[nltk_data]     KUMARI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\NIKITA
[nltk_data]     KUMARI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [200]:
file['review']

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [202]:
vectorizer = TfidfVectorizer(use_idf = True,lowercase = True,max_features=10000)

In [204]:
pickle.dump(vectorizer,open('npl_vectorizer.pkl','wb'))

In [206]:
x = vectorizer.fit_transform(file['review'])
y = file['sentiment']

In [207]:
x.shape

(50000, 10000)

In [208]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [209]:
clf = naive_bayes.MultinomialNB()
clf.fit(x_train,y_train)

MultinomialNB()

In [214]:
accuracy_score(y_test,clf.predict(x_test))*100

85.72999999999999

In [216]:
lr=LogisticRegression()

In [218]:
lr.fit(x_train,y_train)

LogisticRegression()

In [220]:
accuracy_score(y_test,lr.predict(x_test))*100

89.83

In [222]:
pickle.dump(lr,open('reviews_model.pkl','wb'))

In [224]:
rfc = RandomForestClassifier(n_estimators=50, random_state=2,max_depth=5)

In [226]:
rfc.fit(x_train,y_train)

RandomForestClassifier(max_depth=5, n_estimators=50, random_state=2)

In [228]:
accuracy_score(y_test,rfc.predict(x_test))*100

81.08

In [230]:
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [232]:
xgb.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, ...)

In [177]:
accuracy_score(y_test,xgb.predict(x_test))*100

84.44

In [179]:
lr.fit(x,y)

LogisticRegression()

In [181]:
accuracy_score(y_test,lr.predict(x_test))*100

92.31